Start!

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import time

In [3]:
import astropy.units as u
import numpy as np
import pandas as pd
import astropy.constants as c

In [4]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj
from dolan_junction import DolanJunction as junction
import LL_Default_params_LL as dp
from pocket_LL import TransmonPocket1 as pocket


In [5]:
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal import MetalGUI, Dict
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [6]:
gui = MetalGUI(dp.design)
design = dp.design

12:57PM 16s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Populating font family aliases took 195 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 



## Wirebonds

In [7]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [8]:
design.delete_all_components()

In [9]:
top_leftx = -5.025-0.0125
top_lefty = 5.025 +0.0125
left_wb_xs = [1.6, 0.5-0.3]
left_wb_ys = [5-0.5+0.3]
left_wb_ys.append(5-1.6)
for i in range(8):
    new_y = left_wb_ys[-1]-0.85
    new_x = 0.5-0.3
    left_wb_xs.append(new_x)
    left_wb_ys.append(new_y)
# left_wb_ys.append(-5)

left_wb_xs = np.array(left_wb_xs)+top_leftx
left_wb_ys = np.array(left_wb_ys)

orientations_left = [270,0,0,0,0,0,0,0,0,0]

In [10]:
top_rightx = 5.025+0.0125
top_righty = 5.025+0.0125
right_wb_xs = [5-1.6,4.5+0.3]
right_wb_ys = [5.025-0.5+0.3]
right_wb_ys.append(5.025-1.6)
for i in range(8):
    new_y = right_wb_ys[-1]-0.85
    right_wb_ys.append(new_y)
    new_x = 5-0.5+0.3
    right_wb_xs.append(new_x)
right_wb_ys = np.array(right_wb_ys)

orientations_right = [270,180,180,180,180,180,180,180,180,180]

In [11]:
Wirebond_left_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='8um',
                           pad_width='200um',
                           pad_height='120um',
                           pad_gap='50um',
                           taper_height='50um',
                           layer = str(qubit_layer))
                           
# Wirebond_left_opt = Dict(trace_width=design.variables['trace_width'],
#                            trace_gap=design.variables['trace_gap'],
#                            lead_length='25um',
#                            pad_width='150um',
#                            pad_height='200um',
#                            pad_gap='130um',
#                            taper_height='150um')

Place the wirebonds

In [12]:
left_lauch_pads = []
for i in range(10):
    left_lauch_pads.append(LaunchpadWirebond(design,'wb_left'+str(i), options = Dict(pos_x = '{}mm'.format(left_wb_xs[i]),pos_y = '{}mm'.format(left_wb_ys[i]),orientation = orientations_left[i],**Wirebond_left_opt)))
    # gui.rebuild()
    # gui.autoscale()
    # time.sleep(0.1)

In [13]:
right_lauch_pads = []
for i in range(10):
    right_lauch_pads.append(LaunchpadWirebond(design,'wb_right'+str(i), options = Dict(pos_x = '{}mm'.format(right_wb_xs[i]),pos_y = '{}mm'.format(right_wb_ys[i]),orientation = orientations_right[i],**Wirebond_left_opt)))
    # gui.rebuild()
    # gui.autoscale()
    # time.sleep(0.1)

In [14]:
gui.rebuild()

## Qubit pockets

In [15]:
ind_odd = np.array((4, 9, 3, 8, 2, 6, 0, 5, 1, 7))
ind_even = np.array((9, 4, 8, 3, 7, 1, 5, 0, 6, 2))

In [16]:
freq_odd = 3.8+0.2*ind_odd
freq_even = 3.8+0.2*ind_even

In [17]:
pocket_options = dict(
        pos_x = '0mm', 
        pos_y = '0mm', 
        orientation = '0',
        frequency = 5.2,
        guess_path = r'/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_test3.csv',
        coupling_path = '',
        sim = True,
        coord = '(0,0)',
        qubit_layer = qubit_layer,
        junction_layer = junction_layer, 
        ab_layer = ab_layer,
        ab_square_layer = ab_square_layer,
        junction_area_layer = junction_area_layer,
        ab_distance = '70um',
        rotation = 90,
        jj_orientation = 90
        )

In [18]:
qubit_y1 = ['3.8mm','3.65mm','2.70000mm','1.750000mm','0.8000mm',
            '-0.150000mm','-1.10000mm','-2.050000mm','-3.0000mm','-3.950000mm']

First row numbers

In [19]:
qubit_x0 = ['-3.9mm', '-3.1mm', '-2.2mm', 
            '-1.3mm', '-0.4mm', '0.5mm',
            '1.4mm', '2.3mm','3.2mm', '4.05mm']
qubit_y0 = ['4.5mm'] + ['3.8mm']*8 + ['4.4mm']
qubit_angles0 = [90,180,180,180,180,180,180,180,180,270]

### First row

qubits and qubit-qubit connections

In [20]:
pockets = []#p1, p2]
for i in range(10):
    j = i
    pocket_options['pos_x'] = qubit_x0[j]
    pocket_options['pos_y'] = qubit_y0[j]
    pocket_options['frequency'] = freq_even[j]
    pocket_options['coord'] = '({},0)'.format(j)
    pocket_options['rotation'] = qubit_angles0[j]
    p = pocket(design,options = pocket_options)
    pockets.append(p)
    if i ==0:
        pass
    else:
        pockets[i].connect(pockets[i-1])
        # p.connect(pockets[-1])


tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1


In [21]:
gui.rebuild()

Connecting the ends

In [22]:
pockets[-1].connect(right_lauch_pads[0])
pockets[0].connect(left_lauch_pads[0])

### Rows 2-10

In [23]:
distance = 0.93
xs = []
x0 = -4.2
for i in range(10):
    print(x0+distance*i)
    xs.append(str(x0+distance*i)+'mm')

-4.2
-3.27
-2.34
-1.4100000000000001
-0.48
0.4500000000000002
1.38
2.3100000000000005
3.24
4.170000000000001


In [24]:
# xs = ['-4.3mm', '-3.35mm', '-2.4mm', '-1.45mm', '-0.5mm', '0.45mm', '1.4mm', '2.35mm', '3.3mm', '4.25mm']
xs

['-4.2mm',
 '-3.27mm',
 '-2.34mm',
 '-1.4100000000000001mm',
 '-0.48mm',
 '0.4500000000000002mm',
 '1.38mm',
 '2.3100000000000005mm',
 '3.24mm',
 '4.170000000000001mm']

In [25]:
# pockets = []#p1, p2]
for i in range(1,10):
    print(i)
    poc = []
    for j in range(10):
        pocket_options['pos_x'] = xs[j]
        pocket_options['pos_y'] = qubit_y1[i]
        pocket_options['coord'] = '({},{})'.format(j,i)
        if i%2 == 1:
            pocket_options['frequency'] = freq_odd[j]
        else:
            pocket_options['frequency'] = freq_even[j]
        pocket_options['rotation'] = 0 #qubit_angles0[j]
        p = pocket(design,options = pocket_options)
        poc.append(p)
        if j ==0:
           poc[j].connect(left_lauch_pads[i], buffer = 0.0) 
        elif j == 9:
            poc[j].connect(poc[j-1])
            poc[j].connect(right_lauch_pads[i], buffer = 0.0)
        else:
            poc[j].connect(poc[j-1])
    pockets.append(poc)

1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
2
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
3
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1


12:58PM 17s WARNING [check_lengths]: For path table, component=(9,3)CPWwb_right3, key=trace has short segments that could cause issues with fillet. Values in (4-4)  are index(es) in shapely geometry.
12:58PM 17s WARNING [check_lengths]: For path table, component=(9,3)CPWwb_right3, key=cut has short segments that could cause issues with fillet. Values in (4-4)  are index(es) in shapely geometry.


4
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
5
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
6
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
7
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
8
tried1
tried1
tried1
tried1
tried1
tried1


01:00PM 58s ERROR [rebuild]: ERROR in building component name=cpw_(5,8), error='NoneType' object has no attribute 'coords'


tried22
tried1
tried1
tried1
tried1
9
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1
tried1


In [26]:
cut = dp.Cutout(design, 'cutout', options =  Dict(pos_x = str(-(10.15/2-0.3/2)), pos_y = str(10.15/2-0.1/2), layer = qubit_layer))

01:02PM 27s ERROR [__getitem__]: 
 Component name = `cutout`
 Option name    = `helper`


In [27]:
gui.rebuild()

01:08PM 01s WARNING [check_lengths]: For path table, component=(9,3)CPWwb_right3, key=trace has short segments that could cause issues with fillet. Values in (4-4)  are index(es) in shapely geometry.
01:08PM 01s WARNING [check_lengths]: For path table, component=(9,3)CPWwb_right3, key=cut has short segments that could cause issues with fillet. Values in (4-4)  are index(es) in shapely geometry.
01:14PM 36s ERROR [__getitem__]: 
 Component name = `cutout`
 Option name    = `helper`


# Export

In [28]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

In [29]:
a_gds.options.cheese.edge_nocheese = '200um'
a_gds.options.no_cheese.buffer = '16um'
a_gds.options.cheese.cheese_1_radius = '70um'

In [30]:
qubit_layer = '5'
junction_layer = '20'
ab_layer = '31'
ab_square_layer = '30'
junction_area_layer = '60'

In [31]:
a_gds.options['max_points'] = 200

In [32]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = True
a_gds.options['no_cheese']['view_in_file']['main'][1] = True


a_gds.options['fabricate'] = True

In [33]:
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': True,
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': 200,
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '70um',
  'view_in_file': {'main': {1: True,
    '5': True,
    '20': True,
    '31': False,
    '30': False}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '16um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True,
    '5': True,
    '20': True,
    '31': True,
    '30': True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [34]:
a_gds.export_to_gds('export_test_0506_LL.gds')

  df_copy = df_copy.append(orig_row, ignore_index=False)

  df_copy = df_copy.append(orig_row, ignore_index=False)

  df_copy = df_copy.append(orig_row, ignore_index=False)

  df_copy = df_copy.append(orig_row, ignore_index=False)

01:14PM 39s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:14PM 39s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:14PM 39s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:14PM 41s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:14PM 48s WARNING [_import

1

In [35]:
raise SystemExit("Stop right there!")

SystemExit: Stop right there!

  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



In [ ]:
a_gds = design.renderers.gds

In [ ]:
a_gds.options.tolerance = '0.00001'
a_gds.options.cheese.edge_nocheese = '300um'
a_gds.options.no_cheese.buffer = '40um'

In [ ]:
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '70um',
  'view_in_file': {'main': {1: True, 5: True, 2: False, 8: False, 9: False}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '300um'},
 'no_cheese': {'datatype': '99',
  'buffer': '40um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True, 5: True, 2: True, 8: True, 9: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [ ]:
a_gds.options['path_filename'] = 'Customized_Component/Fake_Junctions.GDS'

In [ ]:
a_gds.options.no_cheese.buffer = '0um'

a_gds.options['cheese']['view_in_file']['main'][5] = False
a_gds.options['no_cheese']['view_in_file']['main'][5] = False
a_gds.options['ground_plane'] = True

In [ ]:
q1.options.layer = 1
d.options.layer = 5
gui.rebuild()

NameError: name 'q1' is not defined

In [ ]:
a_gds.options['ground_plane'] = True

In [ ]:

# a_gds.options['short_segments_to_not_fillet'] = True
# a_gds.options['check_short_segments_by_scaling_fillet'] = 1.0
a_gds.export_to_gds('Dolan.gds')#, highlight_qcomponents=['TQ1'])

In [ ]:
a_gds.options

In [ ]:
design.components.keys()

In [ ]:
d = {}


In [ ]:
base = 822.76
dose = 0.8
for j in range(5):
    d['Column' + str(j+1)] = []
    dose = 0.8 + j*0.1
    for i in range(10):
        dose1 = dose + 0.01*i
        d['Column' + str(j+1)].append(base*round(dose1,2))
    
    


In [ ]:
matrix = pd.DataFrame(d)

In [ ]:
matrix.index += 1

In [ ]:
matrix

In [ ]:
design.delete_all_components()

In [ ]:
# gui.rebuild()

In [ ]:
i = 3
lpl = LaunchpadWirebond(design,'wb_left'+str(i), options = Dict(pos_x = '{}mm'.format(left_wb_xs[i]),pos_y = '{}mm'.format(left_wb_ys[i]),orientation = orientations_left[i],**Wirebond_left_opt))
rpr = (LaunchpadWirebond(design,'wb_right'+str(i), options = Dict(pos_x = '{}mm'.format(right_wb_xs[i]),pos_y = '{}mm'.format(right_wb_ys[i]),orientation = orientations_right[i],**Wirebond_left_opt)))
    
gui.rebuild()

In [ ]:
j = 9
pocket_options['pos_x'] = xs[j]
pocket_options['pos_y'] = qubit_y1[i]
pocket_options['coord'] = '({},{})'.format(j,i)
if i%2 == 1:
    pocket_options['frequency'] = freq_odd[j]
else:
    pocket_options['frequency'] = freq_even[j]
pocket_options['rotation'] = 0 #qubit_angles0[j]
p = pocket(gui, design,options = pocket_options)
gui.rebuild()
# poc.append(p)
# if j ==0:
#     poc[j].connect(left_lauch_pads[i], buffer = 0.03) 
# elif j == 9:
#     poc[j].connect(poc[j-1])
#     poc[j].connect(right_lauch_pads[i], buffer = 0.03)
# else:
#     poc[j].connect(poc[j-1])
p.connect(rpr, buffer = 0)

In [ ]:
gui.rebuild()

In [ ]:
p.connect(rpr, buffer = 0)

In [ ]:
dic = {'start': [(4.792, 1.7), (4.78, 1.7), (4.75, 1.67), (4.75, 1.6400000000000001), (4.72, 1.61), (4.7, 1.61), (4.67, 1.6400000000000001), (4.67, 1.72), (4.64, 1.75)], 'end': [(4.78, 1.7), (4.75, 1.67), (4.75, 1.6400000000000001), (4.72, 1.61), (4.7, 1.61), (4.67, 1.6400000000000001), (4.67, 1.72), (4.64, 1.75), (4.6325, 1.75)], 'angle': [180, 225.0, 270, 225.0, 180, 135.0, 90, 135.0, 180], 'length': [0.011999999999999567, 0.04712388980384729, 0.029999999999999805, 0.04712388980384729, 0.019999999999999574, 0.04712388980384729, 0.07999999999999985, 0.04712388980384729, 0.037499999999999645], 'type': ['straight', 'arc', 'straight', 'arc', 'straight', 'arc', 'straight', 'arc', 'straight'], 'clockwise': [0, -1.0, 0, 1.0, 0, 1.0, 0, -1.0, 0]}


In [ ]:
dic['start']

In [ ]:
dic['end']

In [ ]:
dic['type'][8]